# Topic Modeling using doc2vec

## Imports

In [16]:
import numpy as np
import pandas as pd
import collections
import string

import nagisa

from sklearn.cluster import KMeans
from sklearn import utils

import nltk
from nltk.corpus import stopwords

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import multiprocessing

## Data Input and Preprocessing

### Read in Wikipedia data

In [22]:
wiki_df = pd.read_pickle('wiki') 
wiki_df.head()

,article,views,text
0,メインページ,362562853,ようこそ\nウィキペディア - ウィキペディア日本語版 - 百科事典目次\n検索資料・ポータ...
1,星野源,10190763,星野 源（ほしの みなもと、1981年1月28日 - ）は、日本の音楽家、俳優、文筆家。埼玉...
2,真田信繁,9602104,真田 信繁（さなだ のぶしげ）は、安土桃山時代から江戸時代初期にかけての武将、大名。真田昌幸...
3,高橋一生,8571666,高橋 一生（たかはし いっせい、英字表記：Issey Takahashi、1980年12月9...
4,君の名は。,7788879,『君の名は。』（きみのなは、英: Your Name.）は、2016年に公開された新海誠監督...


In [23]:
data = np.array(wiki_df['text'])

### Use nagisa to generate delimited, labeled data

In [6]:
data_split = []
for article in data:
    article_split = re.split('\n|\.|!|！|。', article)
    data_split.append(article_split)

In [8]:
labeled_data = []
# first 1000 due to data/time limitations
for doc_num in range(1000):
    for sent_num in range(len(data_split[doc_num])):
        cur_text = nagisa.tagging(data_split[doc_num][sent_num])
        words = cur_text.words
        postags = cur_text.postags
        char_offset = 0
        for i in range(len(words)):
            labeled_data.append([doc_num, sent_num, char_offset, words[i], postags[i]])
            char_offset += 1
    print("finished " + str(doc_num))
labeled_df = pd.DataFrame(labeled_data)
labeled_df.to_pickle('labeledDataNew.pkl')

finished 0
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13
finished 14
finished 15
finished 16
finished 17
finished 18
finished 19
finished 20
finished 21
finished 22
finished 23
finished 24
finished 25
finished 26
finished 27
finished 28
finished 29
finished 30
finished 31
finished 32
finished 33
finished 34
finished 35
finished 36
finished 37
finished 38
finished 39
finished 40
finished 41
finished 42
finished 43
finished 44
finished 45
finished 46
finished 47
finished 48
finished 49
finished 50
finished 51
finished 52
finished 53
finished 54
finished 55
finished 56
finished 57
finished 58
finished 59
finished 60
finished 61
finished 62
finished 63
finished 64
finished 65
finished 66
finished 67
finished 68
finished 69
finished 70
finished 71
finished 72
finished 73
finished 74
finished 75
finished 76
finished 77
finished 78
finished 79
finished 80
finished 81
finished 82
finished 83
fi

finished 639
finished 640
finished 641
finished 642
finished 643
finished 644
finished 645
finished 646
finished 647
finished 648
finished 649
finished 650
finished 651
finished 652
finished 653
finished 654
finished 655
finished 656
finished 657
finished 658
finished 659
finished 660
finished 661
finished 662
finished 663
finished 664
finished 665
finished 666
finished 667
finished 668
finished 669
finished 670
finished 671
finished 672
finished 673
finished 674
finished 675
finished 676
finished 677
finished 678
finished 679
finished 680
finished 681
finished 682
finished 683
finished 684
finished 685
finished 686
finished 687
finished 688
finished 689
finished 690
finished 691
finished 692
finished 693
finished 694
finished 695
finished 696
finished 697
finished 698
finished 699
finished 700
finished 701
finished 702
finished 703
finished 704
finished 705
finished 706
finished 707
finished 708
finished 709
finished 710
finished 711
finished 712
finished 713
finished 714
finished 715

KeyboardInterrupt: 

In [10]:
labeled_df = pd.DataFrame(labeled_data)

In [11]:
labeled_df.columns = ['docNum','sentNum','charOffset','word','posTag']

In [12]:
labeled_df.to_pickle('labeledDataNew.pkl')

In [4]:
labeled_df = pd.read_pickle('labeledDataNew.pkl')

In [5]:
labeled_df

,docNum,sentNum,charOffset,word,posTag
0,0,0,0,よう,形容詞
1,0,0,1,こそ,助詞
2,0,1,0,ウィキペディア,名詞
3,0,1,1,,空白
4,0,1,2,-,補助記号
...,...,...,...,...,...
9909358,771,1300,18,を,助詞
9909359,771,1300,19,受け,動詞
9909360,771,1300,20,て,助詞
9909361,771,1300,21,回復,名詞


## doc2vec

Roughly following the outline of https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4#:~:text=Doc2vec%20is%20an%20NLP%20tool,of%20scope%20of%20this%20article


### Word Tokenizer on space delimited documents (without sentence context)

The first application of doc2vec will apply the word tokenizer to a dataset of space delimited documents, with sentence delimiters removed and ignored.

In [7]:
# Create space delimited dataset from the nagisa labeled dataframe
space_delimited = []
for i in range(772):
    concat_doc = ' '.join(list(labeled_df[(labeled_df['docNum']==i)]['word']))
    space_delimited.append(concat_doc)

In [9]:
# Create a new dataframe from the space delimited text
dfdf = pd.DataFrame(space_delimited)
dfdf

,0
0,よう こそ ウィキペディア - ウィキペディア 日本 語 版 - 百 科 事...
1,星野 源 ( ほしの みな もと 、 1 9 8 1 年 1 月 2 8 日 -...
2,真田 信繁 ( さなだ のぶしげ ) は 、 安土 桃山 時代 から 江戸 時代 初...
3,高橋 一生 ( たか は し いっせい 、 英字 表記 : Issey Taka...
4,『 君 の 名 は 』 ( きみ の な は 、 英 : Your Name ) は...
...,...
767,VX ガス ( ヴィーエックス ・ ガス 、 V X 、 O - エチル - S - ( 2...
768,上杉 景勝 ( うえ すぎ かげかつ ) は 、 戦国 時代 から 江戸 時代 前期...
769,有賀 さつき ( ありが さつき 、 1 9 6 5 年 9 月 9 日 - ...
770,髙安 晃 ( たかやす あきら 、 1 9 9 0 年 〈 平成 2 年 〉 2 月...


In [55]:
stop = ['あそこ','あっ','あの','あのかた','あの人','あり','あります','ある','あれ','い','いう','います','いる','う','うち','え','お','および','おり','おります','か','かつて','から','が','き','ここ','こちら','こと','この','これ','これら','さ','さらに','し','しかし','する','ず','せ','せる','そこ','そして','その','その他','その後','それ','それぞれ','それで','た','ただし','たち','ため','たり','だ','だっ','だれ','つ','て','で','でき','できる','です','では','でも','と','という','といった','とき','ところ','として','とともに','とも','と共に','どこ','どの','な','ない','なお','なかっ','ながら','なく','なっ','など','なに','なら','なり','なる','なん','に','において','における','について','にて','によって','により','による','に対して','に対する','に関する','の','ので','のみ','は','ば','へ','ほか','ほとんど','ほど','ます','また','または','まで','も','もの','ものの','や','よう','より','ら','られ','られる','れ','れる','を','ん','何','及び','彼','彼女','我々','特に','私','私達','貴方','貴方方''ようこそ','ウィキペディア','ウィキペディア日本語版','百科事典目次','検索資料','空白','補助記号','）', '（', '、', '『', '』', '・','：', '／', '＋', '→', '「', '」', ' ', '-', '/', '\\', '。', '\n', '！']
stop += list(string.punctuation)

# Tokenize the text using nltk; tag documents with integers, as we do not have labeled data
tagged = [TaggedDocument(nltk.word_tokenize(doc), [i]) for i, doc in enumerate(list(dfdf[0]))]

In [57]:
# Set notebook to use available cores
cores = multiprocessing.cpu_count()

In [ ]:
# Build Doc2Vec model
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=200, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tagged])

In [ ]:
# Train Distributed Bag of Words (DBOW) model
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tagged]), total_examples=len(tagged), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [60]:
# Create vector for each document
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [61]:
# Compute feature vectors for all documents
t, x = vec_for_learning(model_dbow, tagged)

In [62]:
# Perform k-means clustering over feature vectors
kmeans = KMeans(init="k-means++", n_clusters=15)
kmeans.fit(x)
labels = kmeans.predict(x)

In [63]:
# Add labels and article titles to dataframe
dfdf['labels'] = labels
dfdf['article'] = list(wiki_df['article'].iloc[0:772])
dfdf

,0,labels,article
0,よう こそ ウィキペディア - ウィキペディア 日本 語 版 - 百 科 事...,1,メインページ
1,星野 源 ( ほしの みな もと 、 1 9 8 1 年 1 月 2 8 日 -...,5,星野源
2,真田 信繁 ( さなだ のぶしげ ) は 、 安土 桃山 時代 から 江戸 時代 初...,2,真田信繁
3,高橋 一生 ( たか は し いっせい 、 英字 表記 : Issey Taka...,13,高橋一生
4,『 君 の 名 は 』 ( きみ の な は 、 英 : Your Name ) は...,9,君の名は。
...,...,...,...
767,VX ガス ( ヴィーエックス ・ ガス 、 V X 、 O - エチル - S - ( 2...,12,VXガス
768,上杉 景勝 ( うえ すぎ かげかつ ) は 、 戦国 時代 から 江戸 時代 前期...,12,上杉景勝
769,有賀 さつき ( ありが さつき 、 1 9 6 5 年 9 月 9 日 - ...,10,有賀さつき
770,髙安 晃 ( たかやす あきら 、 1 9 9 0 年 〈 平成 2 年 〉 2 月...,12,高安晃


In [64]:
# Group article text by cluster
clustered_articles = [list(dfdf[dfdf['labels']==i][0]) for i in range(15)]

In [65]:
# Find the most frequently used words in each cluster of documents

stop = ['あそこ','あっ','あの','あのかた','あの人','あり','あります','ある','あれ','い','いう','います','いる','う','うち','え','お','および','おり','おります','か','かつて','から','が','き','ここ','こちら','こと','この','これ','これら','さ','さらに','し','しかし','する','ず','せ','せる','そこ','そして','その','その他','その後','それ','それぞれ','それで','た','ただし','たち','ため','たり','だ','だっ','だれ','つ','て','で','でき','できる','です','では','でも','と','という','といった','とき','ところ','として','とともに','とも','と共に','どこ','どの','な','ない','なお','なかっ','ながら','なく','なっ','など','なに','なら','なり','なる','なん','に','において','における','について','にて','によって','により','による','に対して','に対する','に関する','の','ので','のみ','は','ば','へ','ほか','ほとんど','ほど','ます','また','または','まで','も','もの','ものの','や','よう','より','ら','られ','られる','れ','れる','を','ん','何','及び','彼','彼女','我々','特に','私','私達','貴方','貴方方''ようこそ','ウィキペディア','ウィキペディア日本語版','百科事典目次','検索資料','空白','補助記号','）', '（', '、', '『', '』', '・','：', '／', '＋', '→', '「', '」', ' ', '-', '/', '\\', '。', '\n', '！', '年', '月', '日', '\u3000', '第', '〜', '者', '人', '的', '〈', '〉', '一', '中']
stop += list(string.punctuation)
stop += list(string.digits)
# had to add lots of extra stop words to eliminate frequently-used, but not meaningful terms

topic_words = []
for topic in range(15):
    common_chars = collections.Counter()
    for article_text in clustered_articles[topic]:
        article_list = article_text.split(' ')
        for word in article_list:
            if word not in stop:
                common_chars[word] += 1
    topic_words.append(common_chars)

In [66]:
# Print most common words in each topic cluster of documents
for topic in range(15):
    print('Most common words in topic ' + str(topic))
    print([topic_words[topic].most_common(10)[i][0] for i in range(10)])
    print()

Most common words in topic 0
['役', '日本', '話', 'テレビ', '後', '編', '声', '版', '党', '登場']

Most common words in topic 1
['後', '巨人', '信長', '家', '役', '日本', 'テレビ', '安倍', '話', '国']

Most common words in topic 2
['後', 'テレビ', '役', '話', '日本', '回', '放送', '時', '登場', '声']

Most common words in topic 3
['役', '後', '戦車', '日本', 'テレビ', '版', '声', '回', 'DVD', 'シリーズ']

Most common words in topic 4
['市', '都市', '指定', '回', '加野', '後', '戦', '万', 'あさ', '屋']

Most common words in topic 5
['役', 'テレビ', '後', '日本', '会', '話', '組', '達', '目', '回']

Most common words in topic 6
['中国', '人民', '国', '中華', '共和', '銀行', '政府', '化', '世界', '部']

Most common words in topic 7
['後', '桐生', 'ライダー', '仮面', '話', 'S', '役', '登場', 'テレビ', '会']

Most common words in topic 8
['テレビ', '日本', 'シリーズ', '戦', '後', '役', '三', '回', '登場', 'ルパン']

Most common words in topic 9
['日本', 'テレビ', '後', '役', '放送', '回', '三島', '生', '話', 'ベトナム']

Most common words in topic 10
['役', '後', 'テレビ', '話', '日本', '声', '時', '版', '目', '”']

Most common words in topic 11
['後', 'テレビ',

Use 5 clusters instead of 15

In [67]:
# Perform k-means clustering over feature vectors
kmeans = KMeans(init="k-means++", n_clusters=5)
kmeans.fit(x)
labels = kmeans.predict(x)

In [68]:
# Add labels and article titles to dataframe
dfdf['labels'] = labels
dfdf['article'] = list(wiki_df['article'].iloc[0:772])
dfdf

,0,labels,article
0,よう こそ ウィキペディア - ウィキペディア 日本 語 版 - 百 科 事...,1,メインページ
1,星野 源 ( ほしの みな もと 、 1 9 8 1 年 1 月 2 8 日 -...,1,星野源
2,真田 信繁 ( さなだ のぶしげ ) は 、 安土 桃山 時代 から 江戸 時代 初...,2,真田信繁
3,高橋 一生 ( たか は し いっせい 、 英字 表記 : Issey Taka...,2,高橋一生
4,『 君 の 名 は 』 ( きみ の な は 、 英 : Your Name ) は...,1,君の名は。
...,...,...,...
767,VX ガス ( ヴィーエックス ・ ガス 、 V X 、 O - エチル - S - ( 2...,1,VXガス
768,上杉 景勝 ( うえ すぎ かげかつ ) は 、 戦国 時代 から 江戸 時代 前期...,4,上杉景勝
769,有賀 さつき ( ありが さつき 、 1 9 6 5 年 9 月 9 日 - ...,4,有賀さつき
770,髙安 晃 ( たかやす あきら 、 1 9 9 0 年 〈 平成 2 年 〉 2 月...,4,高安晃


In [69]:
# Group article text by cluster
clustered_articles = [list(dfdf[dfdf['labels']==i][0]) for i in range(5)]

In [70]:
# Find the most frequently used words in each cluster of documents

stop = ['あそこ','あっ','あの','あのかた','あの人','あり','あります','ある','あれ','い','いう','います','いる','う','うち','え','お','および','おり','おります','か','かつて','から','が','き','ここ','こちら','こと','この','これ','これら','さ','さらに','し','しかし','する','ず','せ','せる','そこ','そして','その','その他','その後','それ','それぞれ','それで','た','ただし','たち','ため','たり','だ','だっ','だれ','つ','て','で','でき','できる','です','では','でも','と','という','といった','とき','ところ','として','とともに','とも','と共に','どこ','どの','な','ない','なお','なかっ','ながら','なく','なっ','など','なに','なら','なり','なる','なん','に','において','における','について','にて','によって','により','による','に対して','に対する','に関する','の','ので','のみ','は','ば','へ','ほか','ほとんど','ほど','ます','また','または','まで','も','もの','ものの','や','よう','より','ら','られ','られる','れ','れる','を','ん','何','及び','彼','彼女','我々','特に','私','私達','貴方','貴方方''ようこそ','ウィキペディア','ウィキペディア日本語版','百科事典目次','検索資料','空白','補助記号','）', '（', '、', '『', '』', '・','：', '／', '＋', '→', '「', '」', ' ', '-', '/', '\\', '。', '\n', '！', '年', '月', '日', '\u3000', '第', '〜', '者', '人', '的', '〈', '〉', '一', '中']
stop += list(string.punctuation)
stop += list(string.digits)
# had to add lots of extra stop words to eliminate frequently-used, but not meaningful terms

topic_words = []
for topic in range(5):
    common_chars = collections.Counter()
    for article_text in clustered_articles[topic]:
        article_list = article_text.split(' ')
        for word in article_list:
            if word not in stop:
                common_chars[word] += 1
    topic_words.append(common_chars)

In [71]:
# Print most common words in each topic cluster of documents
for topic in range(5):
    print('Most common words in topic ' + str(topic))
    print([topic_words[topic].most_common(15)[i][0] for i in range(15)])
    print()

Most common words in topic 0
['市', '都市', '指定', '回', '加野', '後', '戦', '万', 'あさ', '屋', '日本', '大阪', '演', '銀行', '新']

Most common words in topic 1
['後', 'テレビ', '役', '話', '日本', '回', '放送', '時', '声', '登場', '版', '戦', '家', '生', '目']

Most common words in topic 2
['日本', '役', 'テレビ', '後', '版', '回', '話', '放送', '世界', '時', 'ドラマ', '声', '発売', '東京', 'よる']

Most common words in topic 3
['中国', '人民', '国', '中華', '共和', '銀行', '政府', '化', '世界', '部', '党', '経済', '問題', '億', '万']

Most common words in topic 4
['後', '役', 'テレビ', '日本', '話', '版', '回', '時', '声', 'アニメ', '際', '力', '目', '放送', '登場']

